In [ ]:
import datetime
import intake

import pandas as pd
import hvplot.pandas
import param
import panel as pn
pn.extension()

cat = intake.Catalog('https://raw.githubusercontent.com/ContinuumIO/anaconda-package-data/master/catalog/anaconda_package_data.yaml')

today = datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
try:
    monthly = cat.anaconda_package_data_by_month(year=last_month.year, month=last_month.month,
                                                 columns=['pkg_name', 'counts', 'pkg_python', 'pkg_version']).to_dask()
except:
    # if the last month isn't available, get the month before
    month_before = last_month.replace(day=1) - datetime.timedelta(days=1)
    monthly = cat.anaconda_package_data_by_month(year=month_before.year, month=month_before.month,
                                                 columns=['pkg_name', 'counts', 'pkg_python', 'pkg_version']).to_dask()

package_names = list(monthly.pkg_name.unique())

In [ ]:
class PackageExplorer(param.Parameterized):
    package = param.String(default='bokeh')
    version = param.Selector(objects=['*.*'], default='*.*')
    _package = None
    _version = None
    
    @property
    @param.depends('package')
    def data(self):
        if self._package != self.package:
            self._package = self.package
            self._data = monthly[monthly['pkg_name'] == self.package].compute()
            self.update_version()
        return self._data
    
    @property
    @param.depends('package', 'version')
    def percent_df(self):
        if self._package != self.package or self._version != self.version:
            self._version = self.version
            data = self.data
            if self.version != '*.*':
                data = data[data['pkg_version'] == self.version]
            tot = data.groupby('pkg_python').counts.sum()
            pct = tot / tot.sum()
            self._percent_df = pd.DataFrame({'percents': pct, 'downloads': tot})
        return self._percent_df
    
    def update_version(self):
        versions = sorted(list(self.data.pkg_version.unique()), reverse=True)
        self.param.version.objects = ['*.*'] + versions
        self.version = '*.*'

    @param.depends('package', 'version')
    def table(self):
        df = self.percent_df.copy()
        df['percents'] = df.percents.apply(lambda x: f'{x:.2%}')
        return df

    @param.depends('package', 'version')
    def plot(self):
        df = self.percent_df
        return df.hvplot.barh(y='percents', width=300, height=250, 
                              flip_yaxis=True, ylim=(0, 1), xaxis=False)
    
explorer = PackageExplorer(name='')

package_name_widget = pn.widgets.AutocompleteInput(options=package_names, value='bokeh', name='Package name')
pn.Row(
    pn.Column(
        f'## Monthly conda downloads for {last_month.month}-{last_month.year}',
        'This includes downloads from defaults, conda-forge and bioconda',
        pn.panel(explorer.param,
                 widgets={'package': package_name_widget})
    ), 
    explorer.table, 
    pn.Column(
        pn.Spacer(height=40),
        explorer.plot
    )
).servable()